In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
import wrds
import talib as ta
import datetime, time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from tqdm import tqdm_notebook

In [4]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

### deal with crsp return & decile return

In [5]:
crsp = pd.read_csv('data/return.zip', 
                   parse_dates=['date'], infer_datetime_format=True)
crsp[['PERMNO', 'PERMCO']] = crsp[['PERMNO', 'PERMCO']].astype(int)

crsp.RET = crsp.RET.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
crsp.RETX = crsp.RETX.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)

crsp.RET = crsp.RET.fillna(0)
crsp.RETX = crsp.RETX.fillna(0)

crsp = crsp.drop(['PERMCO', 'CUSIP', 'TICKER', 'COMNAM'], axis=1)
crsp.columns = [i.lower() for i in crsp.columns]

crsp = crsp.assign(mv = crsp.prc.abs() * crsp.shrout,
                         ret_p1 = crsp.ret+1,
                         retx_p1 = crsp.retx+1)
crsp.date = pd.to_datetime(crsp.date) + MonthEnd(0)

E:\ProgramData\Anaconda3\envs\quant\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# crsp cpi decile return date 统一
# crsp = crsp.reset_index()

cpi = pd.read_excel('data/CPIAUCSL.xlsx')
cpi.date = pd.to_datetime(cpi.date) + MonthEnd(0)
crsp = pd.merge(crsp, cpi[['date', 'cpi_adjust']], on=['date'])
# crsp = crsp.assign(ret_p1_adj = crsp.ret * crsp.cpi_adjust + 1)
crsp = crsp.assign(mv_adj=crsp.mv / crsp.cpi_adjust)
# crsp = crsp.set_index('date')
crsp = crsp[['date', 'permno', 'mv', 'mv_adj', 'ret_p1', 'retx_p1']]

In [18]:
decile_return = pd.read_csv('data/decile_return.zip',
                           parse_dates=['date'], infer_datetime_format=True)
decile_return.date = pd.to_datetime(decile_return.date) + MonthEnd(0)

In [17]:
decile_return.to_csv('data/decile_data.csv', index=False)

In [28]:
crsp1 = pd.merge(crsp, decile_return[['date', 'permno', 'decret']], 
                 on=['date', 'permno'])

In [ ]:
crsp1.decret = crsp1.decret.fillna(0)
crsp1 = crsp1.assign(dret_p1 = crsp1.decret + 1)
crsp1 = crsp1.drop('decret', axis=1)

In [32]:
crsp1.to_csv('data/return_ensembled.csv', index=False)

### deal with the comp & ccm link

In [ ]:
comp = pd.read_csv('data/fundamental_final_1.1.zip',
                  parse_dates=['datadate'], infer_datetime_format=True)
comp.datadate = pd.to_datetime(comp.datadate) + MonthEnd(0)

comp = comp.drop(['cusip'], axis=1)
comp = comp[comp.fyr > 0]
comp = comp.assign(fystr = comp.datadate.apply(lambda x :  'A-' + x.strftime('%b')),
                  fysm = comp.datadate - MonthBegin(12))

ccm = pd.read_csv('data/ccm_hist.csv')
ccm = ccm.drop('Unnamed: 0', axis=1)
ccm = ccm[((ccm.linktype == 'LU') | (ccm.linktype == 'LC')) & \
   ((ccm.linkprim == 'P') | (ccm.linkprim == 'C'))]
ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

ccm.columns = ['gvkey', 'linkprim', 'liid', 'linktype', 'permno', 'permco', 'linkdt',
       'linkenddt']
ccm1 = ccm[['gvkey', 'permno']]
ccm1.permno = ccm1.permno.astype(int)

comp = pd.merge(left=comp, right=ccm1, on=['gvkey'])
comp = comp[['gvkey', 'datadate', 'fyear', 'fystr', 'fysm', 'permno']]

comp.to_csv('data/fundamental_final_1.2.csv', index=False)

### get decile data

In [ ]:
data_list = []
db = wrds.Connection(wrds_username='kpu2019')
for i in tqdm_notebook(range(1973, 2009)):
    sql_str = """
                select *
                from crsp.ermport1
                where date>='%s-01-01'
                and date<='%s-12-31'
              """ % (i,i)
    
    data_list.append(db.raw_sql(sql_str))
    
db.close()

df = pd.concat(data_list, axis=0)
# save data
# df.to_csv('data/decile_data.csv')